In [1]:
import numpy as np
import copy

In [2]:
def readPuzzle(puzzle, null_char='0'):
    if len(puzzle) != 81:
        raise Exception('puzzle invalid')
    puzzle = np.array(list(puzzle))
    puzzle[puzzle == null_char] = '0'
    puzzle = puzzle.astype(int)
    puzzle = puzzle.reshape((9, 9))
    return puzzle
def readPuzzles(puzzles, null_char='0'):
    return np.array(list(map(lambda puzzle: readPuzzle(puzzle, null_char=null_char), puzzles)))

In [3]:
with open('easy50.txt', 'r') as f:
    easy50 = readPuzzles(f.read().split('\n')[:-1])
with open('hardest.txt', 'r') as f:
    hardest = readPuzzles(f.read().split('\n')[:-1], null_char='.')
with open('sudoku.txt', 'r') as f:
    main = np.array(f.read().split('\n'))
    main = list(main[np.arange(0, main.size) % 10 != 0])
    main = np.array(list(map(lambda line: np.array(list(line)), main)))
    main = main.reshape(50, 9, 9).astype(int)
with open('top95.txt', 'r') as f:
    top95 = readPuzzles(f.read().split('\n')[:-1], null_char='.')

In [4]:
def copyPuzzle(sudoku):
    puzzle = np.empty((9, 9), dtype=object)
    for (i, j), val in np.ndenumerate(sudoku):
        puzzle[i, j] = copy.copy(val)
    return puzzle
def puzzleString(sudoku):
    printstr = ''
    for k in range(9*9 + 9 + 3 + 4):
        printstr += '-'
    printstr += '\n'
    for i in range(0, 9):
        printstr += '| '
        for j in range(0, 9):
            if type(sudoku[i, j]) == list:
                for ch in sudoku[i, j]:
                    printstr += str(ch)
                for k in range(0, 9 - len(sudoku[i, j])):
                    printstr += ' '
            else:
                printstr += '    '
                printstr += str(sudoku[i, j])
                printstr += '    '
            printstr += ' '
            if j % 3 == 2:
                printstr += '| '
        printstr += "\n"
        if i % 3 == 2:
            for k in range(9*9 + 9 + 3 + 4):
                printstr += '-'
            printstr += '\n'
    return printstr

In [5]:
def removeHelper(coord, val, sudoku):
    try:
        sudoku[coord].remove(val)
        if len(sudoku[coord]) == 0:
            return True
    except ValueError:
        pass
    return False

def clearNumbers(i, j, sudoku):
    if type(sudoku[i, j]) != int:
        raise Exception('clearing on non-int')
    val = sudoku[i, j]
    square_i = i // 3 * 3
    square_j = j // 3 * 3
    for k in range(0, 9):
        if type(sudoku[i, k]) == list:
            invalid = removeHelper((i, k), val, sudoku)
            if invalid:
                return False
        if type(sudoku[k, j]) == list:
            invalid = removeHelper((k, j), val, sudoku)
            if invalid:
                return False
        sq_i = square_i + k % 3
        sq_j = square_j + k // 3
        if type(sudoku[sq_i, sq_j]) == list:
            invalid = removeHelper((sq_i, sq_j), val, sudoku)
            if invalid:
                return False
    return sudoku

In [6]:
def findAndClearLone(sudoku, changed):
    for num in range(1, 10):
        for i in range(0, 9):
            box_i = i // 3 * 3
            box_j = i % 3 * 3
            rclear = True
            rcoord = (-1, -1)
            cclear = True
            ccoord = (-1, -1)
            bclear = True
            bcoord = (-1, -1)
            for j in range(0, 9):
                if sudoku[i, j] == num:
                    rclear = False
                elif type(sudoku[i, j]) == list and num in sudoku[i, j]:
                    if rcoord != (-1, -1):
                        rclear = False
                    rcoord = (i, j)

                if sudoku[j, i] == num:
                    cclear = False
                elif type(sudoku[j, i]) == list and num in sudoku[j, i]:
                    if ccoord != (-1, -1):
                        cclear = False
                    ccoord = (j, i)
                
                coord = (box_i + j // 3, box_j + j % 3)
                if sudoku[coord] == num:
                    bclear = False
                elif type(sudoku[coord]) == list and num in sudoku[coord]:
                    if bcoord != (-1, -1):
                        bclear = False
                    bcoord = coord
            if rclear:
                if rcoord == (-1, -1):
                    return (False, True)
                changed = True
                sudoku[rcoord] = [num]
            if cclear:
                if ccoord == (-1, -1):
                    return (False, True)
                changed = True
                sudoku[ccoord] = [num]
            if bclear:
                if bcoord == (-1, -1):
                    return (False, True)
                changed = True
                sudoku[bcoord] = [num]
    return (sudoku, changed)

In [7]:
def checkWin(puzzle):
    if type(puzzle) == bool:
        return False
    for _, val in np.ndenumerate(puzzle):
        if type(val) != int:
            return False
    return True

def solveSudokuRecur(puzzle):
    changed = True
    while changed:
        changed = False
        puzzle, changed = findAndClearLone(puzzle, changed)
        if type(puzzle) == bool:
            return False
        for (i, j), val in np.ndenumerate(puzzle):
            if type(val) == list and len(val) == 1:
                puzzle[i, j] = val[0]
                puzzle = clearNumbers(i, j, puzzle)
                if type(puzzle) == bool:
                    return False
                changed = True
                break

    if checkWin(puzzle):
        return puzzle
    maxLen = 10
    coord = (-1, -1)
    for loc, val in np.ndenumerate(puzzle):
        if type(val) == list and len(val) < maxLen:
            maxLen = len(val)
            coord = loc
    for num in puzzle[coord]:
        newPuzzle = copyPuzzle(puzzle)
        newPuzzle[coord] = num
        newPuzzle = clearNumbers(coord[0], coord[1], newPuzzle)
        newPuzzle = solveSudokuRecur(newPuzzle)
        if checkWin(newPuzzle):
            return newPuzzle
    return False

def solveSudoku(sudoku):
    puzzle = np.empty((9, 9), dtype=object)
    for (i, j), _ in np.ndenumerate(puzzle):
        puzzle[i, j] = np.arange(1, 10).tolist()
    puzzle[sudoku != 0] = sudoku[sudoku!=0]
    for (i, j), val in np.ndenumerate(puzzle):
        if type(val) == int:
            puzzle = clearNumbers(i, j, puzzle)
            if type(puzzle) == bool:
                return False
    return solveSudokuRecur(puzzle)

In [8]:
def solveSudokuVec(sudokus):
    solvedSudokus = []
    for sudoku in sudokus:
        solvedSudokus.append(solveSudoku(sudoku))
    return solvedSudokus

In [9]:
sEasy50 = solveSudokuVec(easy50)
sHardest = solveSudokuVec(hardest)
sMain = solveSudokuVec(main)
sTop95 = solveSudokuVec(top95)

In [ ]:
s